In [ ]:
import torchvision as tv
import numpy as np
import torch
import torch.nn.functional as F

train_dataset1 = tv.datasets.CIFAR10(root="./", train=True,transform=tv.transforms.ToTensor(),download=True)
test_dataset1 = tv.datasets.CIFAR10(root="./", train=False,transform=tv.transforms.ToTensor(),download=True)

train_loader1 = torch.utils.data.DataLoader(dataset = train_dataset1, batch_size = 100, shuffle = True)
test_loader1 = torch.utils.data.DataLoader(dataset = test_dataset1, batch_size = 100, shuffle = False)

MODELNAME1 = "cifar.model"
EPOCH = 10
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

class CIFAR(torch.nn.Module):
  def __init__(self):
    super(CIFAR, self).__init__()
    self.l1 = torch.nn.Linear(3072, 300)
    self.l2 = torch.nn.Linear(300, 300)
    self.l3 = torch.nn.Linear(300, 10)

  def forward(self, x):
    h = F.relu(self.l1(x))
    h = F.relu(self.l2(h))
    y = self.l3(h)
    return y
def train():
  model = CIFAR().to(DEVICE)
  optimizer = torch.optim.Adam(model.parameters())


  for e in range(EPOCH):
    loss = 0
    for images, labels in train_loader1:
      images = images.view(-1, 32*32*3).to(DEVICE)
      labels = labels.to(DEVICE)
      optimizer.zero_grad()
      y = model(images)


      batchloss = F.cross_entropy(y, labels) 
      batchloss.backward()
      optimizer.step()


      loss = loss + batchloss.item()


    print("epoch: {}, loss: {}".format(e, loss))
  torch.save(model.state_dict(),MODELNAME1)


train()

def test():

  correct = 0
  total = len(test_loader1.dataset)
  model = CIFAR().to(DEVICE)
  model.load_state_dict(torch.load(MODELNAME1))
  model.eval()

  for images, labels in test_loader1:
      images = images.view(-1, 32*32*3).to(DEVICE)
      y = model(images)
      y_pred = y.max(dim=1)[1]
      correct = correct + (y_pred == labels).sum()

  print("correct: {}".format(correct))
  print("total: {}".format(total))
  print("accuracy: {}".format(correct/total))

test()

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./cifar-10-python.tar.gz to ./
Files already downloaded and verified
epoch: 0, loss: 922.6601866483688
epoch: 1, loss: 829.5596289634705
epoch: 2, loss: 783.6442151069641
epoch: 3, loss: 758.0551158189774
epoch: 4, loss: 733.6818962097168
epoch: 5, loss: 717.8396317958832
epoch: 6, loss: 699.8789657354355
epoch: 7, loss: 689.1080461740494
epoch: 8, loss: 674.863840341568
epoch: 9, loss: 662.0767595767975
correct: 5044
total: 10000
accuracy: 0.5044000148773193
